# Statistical Methods


In [26]:
import itertools
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.arima_model import ARIMA
# from statsmodels.tsa.holtwinters import ExponentialSmoothing
# from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.tsa.api as smt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
warnings.filterwarnings('ignore')

In [16]:
# Veri Seti
############################

# Atmospheric CO2 from Continuous Air Samples at Mauna Loa Observatory, Hawaii, U.S.A.
# Period of Record: March 1958 - December 2001

In [17]:
data = sm.datasets.co2.load_pandas()


In [18]:
y = data.data
y

,co2
1958-03-29,316.1
1958-04-05,317.3
1958-04-12,317.6
1958-04-19,317.5
1958-04-26,316.4
...,...
2001-12-01,370.3
2001-12-08,370.8
2001-12-15,371.2
2001-12-22,371.3


In [19]:
y = y['co2'].resample('MS').mean()
y

1958-03-01    316.100000
1958-04-01    317.200000
1958-05-01    317.433333
1958-06-01           NaN
1958-07-01    315.625000
                 ...    
2001-08-01    369.425000
2001-09-01    367.880000
2001-10-01    368.050000
2001-11-01    369.375000
2001-12-01    371.020000
Freq: MS, Name: co2, Length: 526, dtype: float64

In [20]:
y = y.fillna(y.bfill())
y

1958-03-01    316.100000
1958-04-01    317.200000
1958-05-01    317.433333
1958-06-01    315.625000
1958-07-01    315.625000
                 ...    
2001-08-01    369.425000
2001-09-01    367.880000
2001-10-01    368.050000
2001-11-01    369.375000
2001-12-01    371.020000
Freq: MS, Name: co2, Length: 526, dtype: float64

In [21]:
train = y[:'1997-12-01']
train

1958-03-01    316.100000
1958-04-01    317.200000
1958-05-01    317.433333
1958-06-01    315.625000
1958-07-01    315.625000
                 ...    
1997-08-01    362.460000
1997-09-01    360.150000
1997-10-01    360.750000
1997-11-01    362.380000
1997-12-01    364.250000
Freq: MS, Name: co2, Length: 478, dtype: float64

In [22]:
test = y['1998-01-01':]
test

1998-01-01    365.340
1998-02-01    366.200
1998-03-01    367.375
1998-04-01    368.525
1998-05-01    369.140
1998-06-01    368.750
1998-07-01    367.600
1998-08-01    365.720
1998-09-01    363.925
1998-10-01    364.320
1998-11-01    365.550
1998-12-01    366.925
1999-01-01    368.120
1999-02-01    368.850
1999-03-01    369.600
1999-04-01    370.975
1999-05-01    370.840
1999-06-01    370.250
1999-07-01    369.000
1999-08-01    366.700
1999-09-01    364.675
1999-10-01    365.140
1999-11-01    366.650
1999-12-01    367.900
2000-01-01    369.020
2000-02-01    369.375
2000-03-01    370.400
2000-04-01    371.540
2000-05-01    371.650
2000-06-01    371.625
2000-07-01    369.940
2000-08-01    367.950
2000-09-01    366.540
2000-10-01    366.725
2000-11-01    368.125
2000-12-01    369.440
2001-01-01    370.175
2001-02-01    371.325
2001-03-01    372.060
2001-04-01    372.775
2001-05-01    373.800
2001-06-01    373.060
2001-07-01    371.300
2001-08-01    369.425
2001-09-01    367.880
2001-10-01

# ARIMA(p, d, q): (Autoregressive Integrated Moving Average)
trend olan serilerde kullanılır ancak mevsimsellik olan serilerde kullanılmaz.

In [28]:
#order p,d ve q değerleri girilir.
#ekrana çıktı kalabalıklığı olmasın diye disp=0 yazılır. (ML deki verbose gibi)
arima_model = ARIMA(train, order=(1, 1, 1)).fit #(disp=0)


In [30]:
arima_model.summary()


AttributeError: 'function' object has no attribute 'summary'

In [ ]:
#48 birim sonraki değeri tahmin et. çıktı kalabalık olmasın benım isteğim 0.index diye sadece 0.index bilgilerini istiyorum.
y_pred = arima_model.forecast(48)[0]


In [ ]:
y_pred = pd.Series(y_pred, index=test.index)


In [ ]:
def plot_co2(train, test, y_pred, title):
    mae = mean_absolute_error(test, y_pred)
    train["1985":].plot(legend=True, label="TRAIN", title=f"{title}, MAE: {round(mae,2)}")
    test.plot(legend=True, label="TEST", figsize=(6, 4))
    y_pred.plot(legend=True, label="PREDICTION")
    plt.show()

In [ ]:
plot_co2(train, test, y_pred, "ARIMA")


- Hyperparameter Optimization (Model Derecelerini Belirleme)
Arıma modellerindeki dışsal parametrelerin optimum değerleirni bulmaya çalışıcağız. bunun için 2 farklı yöntem vardır.
Bunlardan Biri --> AIC & BIC İstatistiklerine Göre Model Derecesini Belirlem
Diğeri --> ACF & PAC grafiklerine göre model derecesini belirleme


In [ ]:
# AIC & BIC İstatistiklerine Göre Model Derecesini Belirleme
#aıc ne kadar düşük o kadar iyi.

In [35]:
#olası 3 parametre değerleri
p = d = q = range(0, 4)
pdq = list(itertools.product(p, d, q))
pdq

[(0, 0, 0),
 (0, 0, 1),
 (0, 0, 2),
 (0, 0, 3),
 (0, 1, 0),
 (0, 1, 1),
 (0, 1, 2),
 (0, 1, 3),
 (0, 2, 0),
 (0, 2, 1),
 (0, 2, 2),
 (0, 2, 3),
 (0, 3, 0),
 (0, 3, 1),
 (0, 3, 2),
 (0, 3, 3),
 (1, 0, 0),
 (1, 0, 1),
 (1, 0, 2),
 (1, 0, 3),
 (1, 1, 0),
 (1, 1, 1),
 (1, 1, 2),
 (1, 1, 3),
 (1, 2, 0),
 (1, 2, 1),
 (1, 2, 2),
 (1, 2, 3),
 (1, 3, 0),
 (1, 3, 1),
 (1, 3, 2),
 (1, 3, 3),
 (2, 0, 0),
 (2, 0, 1),
 (2, 0, 2),
 (2, 0, 3),
 (2, 1, 0),
 (2, 1, 1),
 (2, 1, 2),
 (2, 1, 3),
 (2, 2, 0),
 (2, 2, 1),
 (2, 2, 2),
 (2, 2, 3),
 (2, 3, 0),
 (2, 3, 1),
 (2, 3, 2),
 (2, 3, 3),
 (3, 0, 0),
 (3, 0, 1),
 (3, 0, 2),
 (3, 0, 3),
 (3, 1, 0),
 (3, 1, 1),
 (3, 1, 2),
 (3, 1, 3),
 (3, 2, 0),
 (3, 2, 1),
 (3, 2, 2),
 (3, 2, 3),
 (3, 3, 0),
 (3, 3, 1),
 (3, 3, 2),
 (3, 3, 3)]

In [36]:
def arima_optimizer_aic(train, orders):
    best_aic, best_params = float("inf"), None
    for order in orders:
        try:
            arima_model_result = ARIMA(train, order).fit(disp=0)
            aic = arima_model_result.aic
            if aic < best_aic:
                best_aic, best_params = aic, order
            print('ARIMA%s AIC=%.2f' % (order, aic))
        except:
            continue
    print('Best ARIMA%s AIC=%.2f' % (best_params, best_aic))
    return best_params


In [37]:
best_params_aic = arima_optimizer_aic(train, pdq)


Best ARIMANone AIC=inf


In [ ]:
# Final Model


In [38]:
arima_model = ARIMA(train, best_params_aic).fit(disp=0)


TypeError: fit() got an unexpected keyword argument 'disp'

In [39]:
y_pred = arima_model.forecast(48)[0]


AttributeError: 'function' object has no attribute 'forecast'

In [ ]:
y_pred = pd.Series(y_pred, index=test.index)


In [ ]:
plot_co2(train, test, y_pred, "ARIMA")


# SARIMA(P,D,Q): (Seasonal Autoregressive Integrated Moving-Average)


In [ ]:
#order daki değerler pdq değerlerinin ARIMA değerleridir.(TRENDE İLİŞİN DEĞERLER)
#seasonal_order daki PDQ değerlerinin SARIMA değerleridir. (MEVSİMSELLİĞE İLİŞKİN DEĞERLER)
model = SARIMAX(train, order=(1, 0, 1), seasonal_order=(0, 0, 0, 12))


In [ ]:
sarima_model = model.fit(disp=0)


In [ ]:
#48 adım sonrası için tahmin
y_pred_test = sarima_model.get_forecast(steps=48)


In [ ]:
y_pred = y_pred_test.predicted_mean


In [ ]:
y_pred = pd.Series(y_pred, index=test.index)


In [ ]:
plot_co2(train, test, y_pred, "SARIMA")


In [ ]:
# Hyperparameter Optimization (Model Derecelerini Belirleme)


In [ ]:
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))

In [ ]:
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]


In [ ]:
#aıc değerine göre optimizasyon:
def sarima_optimizer_aic(train, pdq, seasonal_pdq):
    best_aic, best_order, best_seasonal_order = float("inf"), None, None
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                sarimax_model = SARIMAX(train, order=param, seasonal_order=param_seasonal)
                results = sarimax_model.fit(disp=0)
                aic = results.aic
                if aic < best_aic:
                    best_aic, best_order, best_seasonal_order = aic, param, param_seasonal
                print('SARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, aic))
            except:
                continue
    print('SARIMA{}x{}12 - AIC:{}'.format(best_order, best_seasonal_order, best_aic))
    return best_order, best_seasonal_order

In [ ]:
best_order, best_seasonal_order = sarima_optimizer_aic(train, pdq, seasonal_pdq)


In [ ]:
# Final Model


In [ ]:
model = SARIMAX(train, order=best_order, seasonal_order=best_seasonal_order)


In [ ]:
sarima_final_model = model.fit(disp=0)


In [ ]:
y_pred_test = sarima_final_model.get_forecast(steps=48)


In [ ]:
y_pred = y_pred_test.predicted_mean


In [ ]:
y_pred = pd.Series(y_pred, index=test.index)


In [ ]:
plot_co2(train, test, y_pred, "SARIMA")


In [ ]:
# BONUS: MAE'ye Göre SARIMA Optimizasyonu


In [ ]:

p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]


In [ ]:

def sarima_optimizer_mae(train, pdq, seasonal_pdq):
    best_mae, best_order, best_seasonal_order = float("inf"), None, None
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                model = SARIMAX(train, order=param, seasonal_order=param_seasonal)
                sarima_model = model.fit(disp=0)
                y_pred_test = sarima_model.get_forecast(steps=48)
                y_pred = y_pred_test.predicted_mean
                mae = mean_absolute_error(test, y_pred)
                if mae < best_mae:
                    best_mae, best_order, best_seasonal_order = mae, param, param_seasonal
                print('SARIMA{}x{}12 - MAE:{}'.format(param, param_seasonal, mae))
            except:
                continue
    print('SARIMA{}x{}12 - MAE:{}'.format(best_order, best_seasonal_order, best_mae))
    return best_order, best_seasonal_order


In [ ]:
best_order, best_seasonal_order = sarima_optimizer_mae(train, pdq, seasonal_pdq)


In [ ]:
model = SARIMAX(train, order=best_order, seasonal_order=best_seasonal_order)


In [ ]:
sarima_final_model = model.fit(disp=0)


In [ ]:
y_pred_test = sarima_final_model.get_forecast(steps=48)


In [ ]:
y_pred = y_pred_test.predicted_mean


In [ ]:
y_pred = pd.Series(y_pred, index=test.index)


In [ ]:
plot_co2(train, test, y_pred, "SARIMA")


In [ ]:
# Final Model
#tüm veriyi getiriyorum y.
model = SARIMAX(y, order=best_order, seasonal_order=best_seasonal_order)


In [ ]:
sarima_final_model = model.fit(disp=0)


In [ ]:
feature_predict = sarima_final_model.get_forecast(steps=6)


In [ ]:
feature_predict = feature_predict.predicted_mean
